In [1]:
import tensorflow as tf
import tensorflowjs as tfjs
import pandas as pd
import numpy as np
import json

In [2]:
dataTrain = pd.read_csv('./train.csv')
dataTest = pd.read_csv('./test.csv')
dataValidate = pd.read_csv('./validate.csv')
dataTrain.head()

,text,label
0,i won the cricket match,1
1,i ranked first in my class,1
2,i topped in my semesters!,1
3,i didnt feel humiliated,0
4,i am not so happy today,0


In [5]:
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.oov_token = '<shobdobhandarErBahire>'
tokenizer.fit_on_texts(dataTrain['text'])
vocab = tokenizer.word_index
f = open('tokenizer.json', 'w')
f.write(json.dumps(vocab))
f.close()
# print(vocab)
xTrain = tf.keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences(dataTrain['text']))
xTest = tf.keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences(dataTest['text']))
xValidate = tf.keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences(dataValidate['text']))
yTrain = tf.keras.utils.to_categorical(dataTrain['label'])
yTest = tf.keras.utils.to_categorical(dataTest['label'])
yValidate = tf.keras.utils.to_categorical(dataValidate['label'])
vocabSize = len(vocab)
xTrain.shape, yTrain.shape, vocabSize
# xTrain[0]

((16011, 66), (16011, 6), 15220)

In [6]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(vocabSize, 64, input_length=66))
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)))
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dense(6, activation='softmax'))

model.compile(loss='categorical_crossentropy' , optimizer='adam', metrics=['accuracy'])

In [7]:
model.fit(xTrain, yTrain, epochs=15, validation_data=(xValidate, yValidate))

Epoch 1/15
501/501 [==============================] - 15s 15ms/step - loss: 1.1332 - accuracy: 0.5448 - val_loss: 0.6146 - val_accuracy: 0.7740
Epoch 2/15
501/501 [==============================] - 6s 13ms/step - loss: 0.3615 - accuracy: 0.8708 - val_loss: 0.3081 - val_accuracy: 0.9015
Epoch 3/15
501/501 [==============================] - 7s 14ms/step - loss: 0.1593 - accuracy: 0.9454 - val_loss: 0.2618 - val_accuracy: 0.9140
Epoch 4/15
501/501 [==============================] - 6s 13ms/step - loss: 0.0946 - accuracy: 0.9676 - val_loss: 0.3669 - val_accuracy: 0.8920
Epoch 5/15
501/501 [==============================] - 6s 13ms/step - loss: 0.0684 - accuracy: 0.9768 - val_loss: 0.2827 - val_accuracy: 0.9195
Epoch 6/15
501/501 [==============================] - 6s 12ms/step - loss: 0.0475 - accuracy: 0.9849 - val_loss: 0.2733 - val_accuracy: 0.9215
Epoch 7/15
501/501 [==============================] - 6s 12ms/step - loss: 0.0455 - accuracy: 0.9842 - val_loss: 0.2936 - val_accuracy: 0.916

In [8]:
model.evaluate(xTest, yTest)

63/63 [==============================] - 1s 11ms/step - loss: 0.3887 - accuracy: 0.9110


[0.3887183666229248, 0.9110000133514404]

In [10]:
inpt = input('enter the text')
labels_dict = {0:'sadness', 1:'joy', 2:'love', 3:'anger', 4:'fear', 5:'surprise'}
text = tf.keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences([inpt]), padding='pre', maxlen=66)

print(text, model.predict(text), labels_dict[np.argmax(model.predict(text))])

[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0 2970  678    2   24 1477    7  628]] [[0.19357768 0.10633751 0.00724811 0.6083305  0.07997274 0.00453346]] anger


In [7]:
# model.save('ai-model.h5')
tfjs.converters.save_keras_model(model, 'C:\\Users\\007ra\\Documents\\Codes\\AI-ML\\sentiment-analyzer\\AI')